In [1]:
import cmocean as cmo
import gc
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
# Load predictions
fnam = "/home/jbassham/jack/thesis-rough/data/sh/outputs/cnn/weighted/CNNPreds_sh_1992_2020.npz"
data = np.load(fnam, allow_pickle = True)

upred = data['y_pred'][:,0,:,:]
vpred = data['y_pred'][:,1,:,:]

utrue = data['y_true'][:,0,:,:]
vtrue = data['y_true'][:,1,:,:]

# Delete unused variables from memory
del data
gc.collect()

88

In [4]:
# load latitude longitude grid for plotting
fnam = "/home/jbassham/jack/thesis-rough/data/sh/inputs/lat_lon_sh_1992_2020.npz"
data = np.load(fnam, allow_pickle = True)

lat = data['lat']
lon = data['lon']

# Delete unused variables from memory
del data
gc.collect()

22

In [5]:
# Load CNN data split indices and land mask
fnam = "/home/jbassham/jack/thesis-rough/data/sh/cnn-inputs/indices_land_sh_1992_2020.npz"
data = np.load(fnam, allow_pickle = True)

train_idx = data['train_idx']
val_idx = data['val_idx']
test_idx = data['test_idx']
land_mask = data['land_mask']

del data
gc.collect()

44

In [ ]:
# Load time variable

fnam = "/home/jbassham/jack/thesis-rough/data/sh/inputs/time_today_sh_1992_2020.npz"
time = data['time_today']

del data
gc.collect()

In [ ]:
# Load uncertainty, ice velocities
fnam = "/home/jbassham/jack/thesis-rough/data/sh/inputs/inputs_normalized_sh_1992_2020.npz"
data = np.load(fnam, allow_pickle=True)
rt = data['rtn']
uit = data['uitn']
vit = data['vitn']

del data
gc.collect()

In [ ]:
# Crop uncertainty, ice velocities to test indices
rtest = rt[test_idx,:,:]
uitest = uit[test_idx,:,:] 
vitest = vit[test_idx,:,:]
timetest = time[test_idx,:,:]

In [ ]:
# Delete unused variables from memory
del data, timetest, rt, uit, vit
gc.collect()